- 데이터 읽어 들이기 : 농어 길이_두께_높이.csv 파일
     - 농어 무게 데이터는 복사 사용
- 훈련 : 테스트 데이터 = 7: 3 으로 분리
- 특성 늘리기 : 2차원 사용
- 스케일링 : Standard 사용
- 선형회귀모델 훈련
- 테스트 데이터로 예측
- 평균절대오차(MAE) 확인

In [1]:
### 라이브러리 정의
# - 데이터를 읽기 위한 라이브러리
import pandas as pd

# - 시각화 라이브러리
import matplotlib.pyplot as plt

# - 수치 계산 라이브러리
import numpy as np

# - 데이터 가공 라이브러리
from sklearn.model_selection import train_test_split

# - 선형회귀모델 라이브러리
from sklearn.linear_model import LinearRegression

# - 변환기 모델(클래스) 라이브러리 정의
from sklearn.preprocessing import PolynomialFeatures

# - MAE 라이브러리 정의
from sklearn.metrics import mean_absolute_error

# - 데이터 스케일링 라이브러리
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

# - 한글처리
plt.rc("font", family="Malgun Gothic")

# - 마이너스 기호 깨짐 처리
plt.rcParams["axes.unicode_minus"] = False

In [2]:
### 데이터 읽어 들이기
file_path = "./data/03_농어의_길이_높이_두께_데이터.csv"
df = pd.read_csv(file_path)

# 독립변수 데이터 프레임 생성
perch_full = df.to_numpy()

In [3]:
### 종속변수 생성
### 농어의 무게
perch_weight = np.array(
    [5.9, 32.0, 40.0, 51.5, 70.0, 100.0, 78.0, 80.0, 85.0, 85.0, 
     110.0, 115.0, 125.0, 130.0, 120.0, 120.0, 130.0, 135.0, 110.0, 
     130.0, 150.0, 145.0, 150.0, 170.0, 225.0, 145.0, 188.0, 180.0, 
     197.0, 218.0, 300.0, 260.0, 265.0, 250.0, 250.0, 300.0, 320.0, 
     514.0, 556.0, 840.0, 685.0, 700.0, 700.0, 690.0, 900.0, 650.0, 
     820.0, 850.0, 900.0, 1015.0, 820.0, 1100.0, 1000.0, 1100.0, 
     1000.0, 1000.0]
     )

In [4]:
### 데이터 분류하기
# - 분류기준 = 훈련 : 테스트 7 : 3
train_input, test_input, train_target, test_target = train_test_split(
    perch_full, perch_weight, test_size=0.3, random_state=42
)

In [5]:
### 변환기 모델(클래스) 생성
# 2차원으로 특성 생성하기
poly = PolynomialFeatures(include_bias=False)
poly.fit(train_input) # 패턴 찾기

# 찾은 패턴으로 독립변수 데이터 변환
train_poly = poly.transform(train_input)
test_poly = poly.transform(test_input)
print(train_poly.shape, test_poly.shape)

(39, 9) (17, 9)


In [6]:
### StandardScaler 스케일링
ss = StandardScaler()

### 스케일링하기 위한 패턴 찾기
ss.fit(train_poly)

### 찾은 패턴으로 변환(스케일링)
train_scaled = ss.transform(train_poly)
test_scaled = ss.transform(test_poly)

print(train_scaled.shape, train_target.shape)
print(test_scaled.shape, test_target.shape)

(39, 9) (39,)
(17, 9) (17,)


In [7]:
### 훈련 모델 생성
lr = LinearRegression()

### 훈련 시키기
lr.fit(train_scaled, train_target)

### 훈련 및 테스트 데이터 정확도(결정계수) 확인
train_score = lr.score(train_scaled, train_target)
test_score = lr.score(test_scaled, test_target)

### 과적합 여부 해석
print(f"훈련 결정계수 : {train_score}, 테스트 결정계수 : {test_score}, 과적합여부 : {train_score - test_score}")

### 테스트 데이터로 예측
test_pred = lr.predict(test_scaled)
print(f"예측결과 : {test_pred}")

### 예측 결과를 이용해서 평균절대오차(MAE) 확인
mae = mean_absolute_error(test_target, test_pred)
print(f"평균절대오차(MAE) : {mae}")

훈련 결정계수 : 0.9898271546307025, 테스트 결정계수 : 0.97137716006296, 과적합여부 : 0.018449994567742434
예측결과 : [  22.92877835   31.10777487  250.98436388  111.59208216  128.85787135
  779.24558158  304.72417951  176.31471164  916.8961555    98.7200446
 1180.23592257   34.86148711  288.45880297  272.52503942   85.71366627
  120.24574045   59.93605383]
평균절대오차(MAE) : 30.216889590345424


#### 릿지(Ridge) 및 라쏘(Lasso) 모델 사용하기
- 릿지 및 라쏘 모델 특징
    - 과대 및 과소 적합을 조절하는 모델
    - 조절하기 위한 하이퍼 파라미터를 제공하고 있음
    - 하이퍼 파라미터 : 복작도를 조절할 수 있는 파라미터 값을 변경하여 과적합 조정
    - 릿지 및 라쏘에서는 "복잡도"라는 용어를 "규제"라는 용어로 사용함
    - 훈련 정확도는 다소 낮아지는 특징을 가지고 있음
    - 테스트 정확도는 높아지는 경향을 보이는 특징을 가지고 있음
    - 일반화에 최적화된 모델
    - 선형회귀모델 이후 향상된 모델로 많이 사용되고 있는 모델임
<br><br>

- 사용 방법
    - 사용 패티지 : sklearn.linear_model
    - 사용 모델(클래스) : Ridge, Lasso
    - 사용 함수 : 훈련(fit) 및 예측(predict) 등 기존과 동일한 함수 사용

#### 릿지(Ridge) 모델

In [8]:
### 라이브러리 정의
from sklearn.linear_model import Ridge

In [9]:
### 모델 생성
# 변수명 : ridge
ridge = Ridge()

ridge

Ridge()

In [10]:
### 훈련시키기
ridge.fit(train_scaled, train_target)

Ridge()

In [11]:
### 훈련 및 테스트 정확도 확인
# - 과적합 여부 확인
train_score = ridge.score(train_scaled, train_target)
test_score = ridge.score(test_scaled, test_target)

print("과적합 여부 확인")
print(f"훈련 결정계수 : {train_score}, 테스트 결정계수 : {test_score}, 과적합여부 : {train_score - test_score}")

과적합 여부 확인
훈련 결정계수 : 0.9849041294689239, 테스트 결정계수 : 0.9845173591615219, 과적합여부 : 0.0003867703074019735


In [12]:
### 테스트 데이터로 예측
test_pred = ridge.predict(test_scaled)

print(f"예측결과 : {test_pred}")

예측결과 : [ -61.93263901   69.38828574  279.34790525  140.45052142  137.16491873
  796.87105093  335.18396118  207.78335133  836.88122091  118.54861871
 1083.37235419   26.09003783  294.35909149  347.40000282   74.06227077
  130.15404326   70.08272221]


In [13]:
### 평균절대오차(MAE) 확인
mae = mean_absolute_error(test_target, test_pred)

print(f"평균절대오차(MAE) : {mae}")

평균절대오차(MAE) : 29.25391147401816


#### 라쏘(Lasso) 모델

In [14]:
### 라이브러리 정의
from sklearn.linear_model import Lasso

In [15]:
### 모델 생성
# - 변수명 : lasso
lasso = Lasso()

lasso

Lasso()

In [16]:
### 모델 훈련
lasso.fit(train_scaled, train_target)

Lasso()

In [17]:
### 훈련 및 테스트 정확도 확인
# - 과적합 여부 확인
train_score = lasso.score(train_scaled, train_target)
test_score = lasso.score(test_scaled, test_target)

print("과적합 여부 확인")
print(f"훈련 결정계수 : {train_score}, 테스트 결정계수 : {test_score}, 과적합여부 : {train_score - test_score}")

과적합 여부 확인
훈련 결정계수 : 0.9861305259897016, 테스트 결정계수 : 0.98632202810554, 과적합여부 : -0.00019150211583840449


In [18]:
### 테스트 데이터 예측
test_pred = lasso.predict(test_scaled)

print(f"예측결과 : {test_pred}")

예측결과 : [ -43.70669743   70.41454264  274.03291416  138.14023972  136.1833532
  795.22130424  328.60470097  204.24315928  822.62808732  118.11124527
 1075.40813513   28.64520876  292.98922415  341.90442243   75.48401454
  129.1560788    71.01982925]


In [19]:
### 평균절대오차(MAE) 확인
mae = mean_absolute_error(test_target, test_pred)

print(f"평균절대오차(MAE) : {mae}")

평균절대오차(MAE) : 26.878384450173353


#### 릿지 및 라쏘 하이퍼 파라미터 튜닝(규제)

- 규제 - 하이퍼 파라미터
    - 모델 성능(정확도 및 과적합여부)에 영향을 미치는 파라미터
    - 규제 = 규제강도 = 강도 = 복잡도 모두 같은 의미
    - 규제 하이퍼 파라미터 이름 : alpha
    - 규제 값의 범위 : 0.001 ~ 100 사이의 값 중에 1개 사용
    - 규제 기본값 (default) : 1(생략가능)

- 규제 값의 성격
    - alpha 값이 작을 수록 훈련 정확도는 낮아지는 특징이 있음, 과대적합 모델에 적합
    - alpha 값이 높을 수록 훈련 정확도는 높아지는 특징이 있음, 과소적합 모델에 적합
    - 실제 값이 작고 높음에 따라서 확인을 해봐야 정확함

In [20]:
### Ridge 모델 생성
# - 변수명 : model
model = Ridge(alpha=0.1)

### 모델 훈련
model.fit(train_scaled, train_target)

### 훈련 및 테스트 정확도 확인
# - 과적합 여부 확인
train_score = model.score(train_scaled, train_target)
test_score = model.score(test_scaled, test_target)

print("과적합 여부 확인")
print(f"훈련 결정계수 : {train_score}, 테스트 결정계수 : {test_score}, 과적합여부 : {train_score - test_score}")

### 테스트 데이터 예측
test_pred = model.predict(test_scaled)

print(f"예측결과 : {test_pred}")

### 평균절대오차(MAE) 확인
mae = mean_absolute_error(test_target, test_pred)

print(f"평균절대오차(MAE) : {mae}")

과적합 여부 확인
훈련 결정계수 : 0.988278016139003, 테스트 결정계수 : 0.9868237771849514, 과적합여부 : 0.001454238954051612
예측결과 : [  34.22542574   74.32747636  258.94188823  132.89237712  132.78422161
  797.65328833  314.99465749  191.55605034  834.71671627  114.21692134
 1112.22606813   50.35370888  276.94961625  318.9311116    84.53480179
  126.86011452   78.52600275]
평균절대오차(MAE) : 20.233828045889517


In [21]:
### Lasso 모델 생성
# - 변수명 : model
model = Lasso(alpha=0.1)

### 모델 훈련
model.fit(train_scaled, train_target)

### 훈련 및 테스트 정확도 확인
# - 과적합 여부 확인
train_score = model.score(train_scaled, train_target)
test_score = model.score(test_scaled, test_target)

print("과적합 여부 확인")
print(f"훈련 결정계수 : {train_score}, 테스트 결정계수 : {test_score}, 과적합여부 : {train_score - test_score}")

### 테스트 데이터 예측
test_pred = model.predict(test_scaled)

print(f"예측결과 : {test_pred}")

### 평균절대오차(MAE) 확인
mae = mean_absolute_error(test_target, test_pred)

print(f"평균절대오차(MAE) : {mae}")

과적합 여부 확인
훈련 결정계수 : 0.9883448062768178, 테스트 결정계수 : 0.9857016019582313, 과적합여부 : 0.002643204318586445
예측결과 : [  63.5977781    86.93900287  256.4242922   133.00575934  132.51622737
  798.88899581  310.18656352  191.55295644  830.27249954  120.38485642
 1109.5480759    57.59416429  269.26020076  320.64188511   85.18138693
  126.19280913   83.96655682]
평균절대오차(MAE) : 21.295562116910823


c:\Users\Administrator\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.443e+03, tolerance: 4.887e+02
  model = cd_fast.enet_coordinate_descent(
